##### Required Imports and Helper functions taken from the coursera project

In [201]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
from pydub import AudioSegment
import numpy as np
import matplotlib.pyplot as plt
from pydub.playback import play
import librosa as lr

# Calculate and plot spectrogram for a wav audio file
def graph_spectrogram(wav_file):
    rate, data = get_wav_info(wav_file)
    nfft = 200 # Length of each window segment
    fs = 8000 # Sampling frequencies
    noverlap = 120 # Overlap between windows
    nchannels = data.ndim
    if nchannels == 1:
        pxx, freqs, bins, im = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, freqs, bins, im = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
    return pxx

# Load a wav file
def get_wav_info(wav_file):
    data, rate = lr.load(path=wav_file,sr=16000)
    return rate, data

# Used to standardize volume of audio clip
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

# Load raw audio files for speech synthesis
def load_raw_audio():
    activates = []
    backgrounds = []
    negatives = []
    for filename in os.listdir("positives"):
        if filename.endswith("wav"):
            activate = AudioSegment.from_wav("positives/"+filename)
            activates.append(activate)
    for filename in os.listdir("backgrounds"):
        if filename.endswith("wav"):
            background = AudioSegment.from_wav("backgrounds/"+filename)
            backgrounds.append(background)
    for filename in os.listdir("negatives"):
        if filename.endswith("wav"):
            negative = AudioSegment.from_wav("negatives/"+filename)
            negatives.append(negative)
    return positives, negatives, backgrounds

In [12]:
import IPython.display as ipd
#ipd.Audio("positives/1.wav")
#ipd.Audio("negatives/1.wav")
ipd.Audio("backgrounds/white_noise.wav")

### Converting the background noises to 10s clips

In [13]:
ac_noise=AudioSegment.from_wav("backgrounds/ac_noise.wav")

In [14]:
time=10*1000
ac_noise_clipped=ac_noise[:time]

In [15]:
ac_noise_clipped.export("backgrounds/ac_noise_clipped.wav",format="wav")

<_io.BufferedRandom name='backgrounds/ac_noise_clipped.wav'>

In [16]:
pink_noise=AudioSegment.from_wav("backgrounds/pink_noise.wav")

In [17]:
pink_noise_clipped=pink_noise[:time]

In [18]:
pink_noise_clipped.export("backgrounds/pink_noise_clipped.wav",format="wav")

<_io.BufferedRandom name='backgrounds/pink_noise_clipped.wav'>

In [19]:
white_noise=AudioSegment.from_wav("backgrounds/white_noise.wav")

In [20]:
white_noise_clipped=white_noise[:time]

In [21]:
white_noise_clipped.export("backgrounds/white_noise_clipped.wav",format="wav")

<_io.BufferedRandom name='backgrounds/white_noise_clipped.wav'>

In [22]:
doing_the_dishes_noise=AudioSegment.from_wav("backgrounds/doing_the_dishes_noise.wav")

In [23]:
doing_the_dishes_noise_clipped=doing_the_dishes_noise[:time]

In [24]:
doing_the_dishes_noise_clipped.export("backgrounds/doing_the_dishes_noise_clipped.wav",format="wav")

<_io.BufferedRandom name='backgrounds/doing_the_dishes_noise_clipped.wav'>

# The original sampling rates for the noises are:
ac_noise=> 48000
doing_the_dishes_noise=>16000
pink_noise=>16000
white_noise=>16000

In [25]:
rate,audio=get_wav_info("backgrounds/white_noise.wav")

In [26]:
rate,audio.shape

(16000, (960000,))

there is a small confusion! the audio part of the ac_noise read using wavfile.read from scipy gives a two dimensional array but if its read using librosa that problem is done, still the no of samples in the ac_noise and ac_noise_clipped is different

In [27]:
rate,audio=get_wav_info("backgrounds/ac_noise.wav")

In [28]:
audio.shape,rate

((623616, 2), 48000)

###          







### checking if the clipping is proper and analysing various rates and audio files

In [29]:
rate_doing_the_dishes_noise_clipped,audio_doing_the_dishes_noise_clipped=get_wav_info("backgrounds/white_noise_clipped.wav")

In [30]:
ipd.Audio("backgrounds/doing_the_dishes_noise_clipped.wav")

In [31]:
rate_doing_the_dishes_noise_clipped,audio_doing_the_dishes_noise_clipped.shape

(16000, (160000,))

 ###                                                                                                                                                                                                                                                                                                                                                                                

In [33]:
ipd.Audio("backgrounds/doing_the_dishes_noise_clipped.wav")

In [34]:
audio3,rate3=lr.load("backgrounds/doing_the_dishes_noise_clipped.wav")

In [35]:
audio3.shape

(220500,)

In [36]:
rate3

22050

In [199]:
rate4,audio4=get_wav_info("positives/42.wav")

In [200]:
rate4,audio4.shape

(48000, (41760, 2))

In [210]:
rate5,data5=get_wav_info("backgrounds/ac_noise_clipped.wav")

In [146]:
ipd.Audio("positives/42.wav")

In [214]:
print(rate5,data5.shape)

16000 (160000,)


In [215]:
ipd.Audio(data=data5,rate=rate5)

## Now all the audio files are ready to be used, you can get the rate and the audio array by just calling the function get_wav_info

In [312]:
rate3,audio3=get_wav_info("negatives/30.wav")

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\SUMAN SVD\\trigger-word\\negatives\\30.wav'

In [313]:
print(rate3,audio3.shape)

16000 (16000,)


In [311]:
ipd.Audio(data=audio3,rate=rate3)